In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
DATA_DIR = '/content/drive/Shareddrives/Intelligent System - Assignment /Data/'

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import nltk
import string
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords, wordnet as wn
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim

In [ ]:
gensim.__version__

'3.6.0'

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
data = pd.read_csv(DATA_DIR+'guardians-news/combined_filtered.csv')
data.head()

,Unnamed: 0,apiUrl,fields.bodyText,id,isHosted,pillarId,pillarName,sectionId,sectionName,type,webPublicationDate,webTitle,webUrl,filtered_bodyText
0,0,https://content.guardianapis.com/technology/20...,A day after Barack Obama announced tough new s...,technology/2016/dec/31/russia-hacking-malware-...,False,pillar/news,News,technology,Technology,article,2016-12-31T16:34:02Z,US-Russia tensions rise as malware found at Ve...,https://www.theguardian.com/technology/2016/de...,"['day', 'barack', 'obama', 'announced', 'tough..."
1,1,https://content.guardianapis.com/technology/20...,A journalist was temporarily banned from Faceb...,technology/2016/dec/30/facebook-temporary-ban-...,False,pillar/news,News,technology,Technology,article,2016-12-30T21:22:06Z,Facebook temporarily bans author after he call...,https://www.theguardian.com/technology/2016/de...,"['journalist', 'temporarily', 'banned', 'faceb..."
2,2,https://content.guardianapis.com/technology/20...,Technology? Bah humbug: “I think we ought to g...,technology/2016/dec/30/donald-trump-technology...,False,pillar/news,News,technology,Technology,article,2016-12-30T08:30:28Z,Donald Trump is technology's befuddled (but da...,https://www.theguardian.com/technology/2016/de...,"['technology', 'bah', 'humbug', 'think', 'ough..."
3,3,https://content.guardianapis.com/technology/20...,While there were some good things in technolog...,technology/2016/dec/30/detonating-devices-and-...,False,pillar/news,News,technology,Technology,article,2016-12-30T07:00:26Z,Exploding phones and Snapchat clones: the bigg...,https://www.theguardian.com/technology/2016/de...,"['good', 'things', 'technology', 'released', '..."
4,4,https://content.guardianapis.com/technology/20...,The US Department of Homeland Security (DHS) a...,technology/2016/dec/29/fbi-dhs-russian-hacking...,False,pillar/news,News,technology,Technology,article,2016-12-29T22:19:53Z,FBI and Homeland Security detail Russian hacki...,https://www.theguardian.com/technology/2016/de...,"['us', 'department', 'homeland', 'security', '..."


In [ ]:
data.shape

(52900, 14)

In [ ]:
def preprocessing(inp):
  text = str(inp)
  tokens = [re.sub(r'[^a-z|A-Z]', ' ', word.strip().lower()) for word in text.split()]

  return " ".join(tokens)

In [ ]:
data['fields.bodyText'][0]

'A day after Barack Obama announced tough new sanctions over what intelligence agencies believe to be Russian attempts to influence the presidential election in favour of Donald Trump, US officials said computer code linked to Russian-sponsored hackers had been detected in a computer at a Vermont electric utility. The municipally run Burlington electric department confirmed on Friday that it had found, in a laptop not connected to grid systems, malware code used in Grizzly Steppe, the name the Department of Homeland Security (DHS) and FBI have applied to a Russian campaign linked to cyber-attacks on the Democratic National Committee and other political organizations. The Washington Post first reported the discovery. On Thursday, the day on which Obama announced the new sanctions, the DHS and the FBI published a report detailing what they called Russia’s “ongoing campaign of cyber-enabled operations directed at the US government and its citizens”. After the discovery in Vermont, however

In [ ]:
preprocessing(preprocessing(data.loc[0, 'fields.bodyText']))

'a day after barack obama announced tough new sanctions over what intelligence agencies believe to be russian attempts to influence the presidential election in favour of donald trump us officials said computer code linked to russian sponsored hackers had been detected in a computer at a vermont electric utility the municipally run burlington electric department confirmed on friday that it had found in a laptop not connected to grid systems malware code used in grizzly steppe the name the department of homeland security dhs and fbi have applied to a russian campaign linked to cyber attacks on the democratic national committee and other political organizations the washington post first reported the discovery on thursday the day on which obama announced the new sanctions the dhs and the fbi published a report detailing what they called russia s ongoing campaign of cyber enabled operations directed at the us government and its citizens after the discovery in vermont however officials said

In [ ]:
data['text'] = data['webTitle'] + ' ' + data['fields.bodyText']
data['text'] = data['text'].apply(preprocessing).apply(preprocessing)
data['text']

In [ ]:
data[['text', 'sectionName']].to_csv(DATA_DIR+'guardians-news/processed.csv', index=False)

In [ ]:
# text = data['text'].values.tolist()
text = pd.read_csv(DATA_DIR+'guardians-news/processed.csv')
text.head()

,text,sectionName
0,us russia tensions rise as malware found at ve...,Technology
1,facebook temporarily bans author after he call...,Technology
2,donald trump is technology s befuddled but dan...,Technology
3,exploding phones and snapchat clones the bigge...,Technology
4,fbi and homeland security detail russian hacki...,Technology


In [ ]:
X = text['text'].values

In [ ]:
LE = LabelEncoder()
y = pd.get_dummies(text['sectionName'])
display(y)
y = y.values

,Business,Culture,Politics,Sport,Technology
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1
...,...,...,...,...,...
52895,0,1,0,0,0
52896,0,1,0,0,0
52897,0,1,0,0,0
52898,0,1,0,0,0


## 1. Doc2Vec + CNN

In [ ]:
max_epochs = 5
vec_size = 50

In [ ]:
tagged_data = [TaggedDocument(words=str(_d), tags=[str(i)]) for i, _d in enumerate(text.values.tolist())]

In [ ]:
model = Doc2Vec(vector_size=vec_size,
                min_count=5,
                dm=1,
                window=3,
                epochs=10)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.epochs)
    # decrease the learning rate
    model.alpha -= 0.002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

In [ ]:
t = text.iloc[:, 0]

In [ ]:
model = Doc2Vec.load("/content/drive/Shareddrives/Intelligent System - Assignment /Models/d2v.model")
v1 = model.infer_vector(t[0])
v1

array([-2.6276188 , -5.6405334 ,  2.98892   ,  0.97198796, -1.6048871 ,
        1.911141  ,  2.7014494 ,  1.7710059 ,  5.6246133 ,  0.78495574,
       -2.7140965 , -0.11351073,  3.9329495 , -0.5172994 , -2.0354435 ,
        2.3710284 , -4.0948057 ,  5.772133  ,  6.711762  ,  3.9790463 ,
        2.160626  ,  5.6973104 , -0.7618695 ,  2.0522797 , -1.2629738 ,
        6.987063  ,  1.3298229 , 11.656668  ,  0.9680128 ,  6.176747  ,
        0.6369994 , -3.7222776 ,  1.4766474 , -2.6831946 ,  4.694322  ,
        0.6379044 , -5.1577053 ,  2.076275  ,  1.6435039 ,  0.23569146,
       -1.2522709 ,  1.6628609 , -0.88996214, -3.284606  ,  1.8125341 ,
        4.0799074 , -0.4574275 , -3.5707932 ,  4.982184  , -1.4823871 ],
      dtype=float32)

In [ ]:
input_matrix = np.zeros((text.shape[0], vec_size))

In [ ]:
for i, d in enumerate(t):
  input_matrix[i] = model.infer_vector(str(d).split())

In [ ]:
X = input_matrix

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((42320, 50), (10580, 50), (42320, 5), (10580, 5))

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv1D(16, kernel_size=3, activation='relu', input_shape=(50, 1)),
  tf.keras.layers.MaxPool1D(pool_size=2),
  tf.keras.layers.Conv1D(32, kernel_size=3, activation='relu'),
  tf.keras.layers.MaxPool1D(pool_size=2),
  tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 48, 16)            64        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 24, 16)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 22, 32)            1568      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 11, 32)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 9, 64)             6208      
                                                                 
 flatten (Flatten)           (None, 576)               0

In [ ]:
model.fit(X_train[:, :, np.newaxis], y_train, epochs=100, batch_size=256)

Epoch 1/100
166/166 [==============================] - 14s 8ms/step - loss: 0.4224 - accuracy: 0.8473
Epoch 2/100
166/166 [==============================] - 1s 8ms/step - loss: 0.2269 - accuracy: 0.9207
Epoch 3/100
166/166 [==============================] - 1s 8ms/step - loss: 0.2042 - accuracy: 0.9287
Epoch 4/100
166/166 [==============================] - 1s 8ms/step - loss: 0.1886 - accuracy: 0.9337
Epoch 5/100
166/166 [==============================] - 1s 8ms/step - loss: 0.1797 - accuracy: 0.9354
Epoch 6/100
166/166 [==============================] - 1s 8ms/step - loss: 0.1670 - accuracy: 0.9412
Epoch 7/100
166/166 [==============================] - 1s 8ms/step - loss: 0.1587 - accuracy: 0.9434
Epoch 8/100
166/166 [==============================] - 1s 8ms/step - loss: 0.1506 - accuracy: 0.9458
Epoch 9/100
166/166 [==============================] - 1s 8ms/step - loss: 0.1453 - accuracy: 0.9474
Epoch 10/100
166/166 [==============================] - 1s 8ms/step - loss: 0.1402 - accur

In [ ]:
y_pred = model.predict(X_test[:, :, np.newaxis])
y_pred

array([[2.4088286e-04, 6.9468438e-06, 3.7628575e-10, 9.9908280e-01,
        6.6937023e-04],
       [4.6119953e-06, 1.2350834e-08, 9.9999523e-01, 8.1363282e-08,
        2.0149486e-10],
       [7.0982509e-07, 2.8870781e-08, 9.9999785e-01, 3.5684724e-09,
        1.3959962e-06],
       ...,
       [9.9952734e-01, 2.2082179e-06, 4.6870831e-04, 2.4329427e-07,
        1.4034614e-06],
       [8.0014655e-29, 1.0324048e-30, 1.2636048e-34, 1.0000000e+00,
        3.4619857e-26],
       [5.4800557e-07, 3.1662286e-12, 4.2848464e-16, 9.9999940e-01,
        7.0380114e-19]], dtype=float32)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      2467
           1       0.86      0.83      0.84       500
           2       0.94      0.94      0.94      2465
           3       0.98      0.99      0.98      4103
           4       0.89      0.89      0.89      1045

    accuracy                           0.94     10580
   macro avg       0.92      0.91      0.91     10580
weighted avg       0.94      0.94      0.94     10580



In [ ]:
model.evaluate(X_test[:, :, np.newaxis], y_test)

331/331 [==============================] - 2s 4ms/step - loss: 0.3010 - accuracy: 0.9415


[0.30095088481903076, 0.9414933919906616]

In [ ]:
# Analysis on wrong predictions
wrong_indices = np.argmax(y_test, axis=1) != np.argmax(y_pred, axis=1)
wrong_test = y_test[wrong_indices]
wrong_pred = y_pred[wrong_indices]
for i in range(min(wrong_test.shape[0], 100)):
  print('Wrong sample {}'.format(i))
  print(wrong_test[i])
  print(wrong_pred[i])

Wrong sample 0
[0 0 1 0 0]
[7.3200017e-01 3.2740631e-05 2.4951208e-01 8.8858324e-05 1.8366119e-02]
Wrong sample 1
[0 0 0 0 1]
[1.7057976e-08 9.9777114e-01 2.9149819e-13 2.1014732e-04 2.0186710e-03]
Wrong sample 2
[0 0 0 1 0]
[0.04498796 0.41364807 0.11395615 0.41304767 0.01436014]
Wrong sample 3
[0 0 0 1 0]
[1.3406356e-01 8.3055288e-01 1.4625977e-06 3.5381161e-02 8.7788453e-07]
Wrong sample 4
[0 0 0 0 1]
[9.1494155e-01 1.7921295e-05 3.0085500e-04 1.4976901e-07 8.4739514e-02]
Wrong sample 5
[0 0 0 1 0]
[1.7358299e-04 9.7873974e-01 5.1408616e-08 2.0826502e-02 2.6007046e-04]
Wrong sample 6
[0 0 1 0 0]
[9.8678106e-01 8.3355270e-03 3.9851349e-03 4.0555323e-04 4.9273641e-04]
Wrong sample 7
[1 0 0 0 0]
[0.03624619 0.3296292  0.20626764 0.4151252  0.01273174]
Wrong sample 8
[1 0 0 0 0]
[2.1912609e-03 1.7576454e-08 9.9780041e-01 5.4411703e-10 8.3466193e-06]
Wrong sample 9
[0 0 0 1 0]
[1.7442834e-02 9.6639854e-01 2.1764909e-05 1.8404309e-03 1.4296431e-02]
Wrong sample 10
[0 0 0 1 0]
[1.3293770e-

In [ ]:
model.save_weights('./model/doc2vec_checkpoint')

## 2. GloVe + CNN

In [ ]:
X = text.iloc[:, 0].astype(str)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train

8339     sadiq khan i don t want exemption from ignoran...
43538    dyson heppell named new essendon captain for a...
1895     uber s human drivers are mapping out the syste...
41778    salford s title push continues with convincing...
37858    andy murray beats jo o sousa at australian ope...
                               ...                        
48321    county cricket yorkshire rally after first inn...
6448     nick clegg i stopped talking to michael gove d...
151      oculus miffed when vr is so immersive you fall...
25595    eurozone reports sluggish growth failing to me...
14592    theresa may s draft brexit deal what is it and...
Name: text, Length: 42320, dtype: object

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
words_to_index = tokenizer.word_index

In [ ]:
def read_glove_vector(glove_vec):
    with open(glove_vec, 'r', encoding='UTF-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            w_line = line.split()
            curr_word = w_line[0]
            word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)
    return word_to_vec_map

In [ ]:
word_to_vec_map = read_glove_vector('/content/drive/Shareddrives/Intelligent System - Assignment /Glove/glove.6B.100d.txt')

In [ ]:
maxLen = 500
embed_vector_len = 100
vocab_len = len(words_to_index) + 1

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        emb_matrix[index, :] = embedding_vector

In [ ]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

X_train_indices = tf.keras.preprocessing.sequence.pad_sequences(X_train_indices, maxlen=maxLen, padding='post')

In [ ]:
def glove_cnn(filters = 30, input_dim = None, output_dim=300, max_length = None, emb_matrix = None):
  
    # Channel 1D CNN
    input = tf.keras.Input(shape=(max_length,))
    embedding = tf.keras.layers.Embedding(input_dim=input_dim, 
                            output_dim=output_dim, 
                            input_length=max_length, 
                            input_shape=(max_length, ),
                            # Assign the embedding weight with word2vec embedding marix
                            weights = [emb_matrix],
                            # Set the weight to be not trainable (static)
                            trainable = False)(input)
    conv1 = tf.keras.layers.Conv1D(filters=filters, kernel_size=2, activation='relu', 
                   kernel_constraint= tf.keras.constraints.MaxNorm(max_value=3, axis=[0,1]))(embedding)
    pool1 = tf.keras.layers.GlobalMaxPool1D()(conv1)
    conv2 = tf.keras.layers.Conv1D(filters=filters, kernel_size=3, activation='relu', 
                   kernel_constraint= tf.keras.constraints.MaxNorm(max_value=3, axis=[0,1]))(embedding)
    pool2 = tf.keras.layers.GlobalMaxPool1D()(conv2)
    conv3 = tf.keras.layers.Conv1D(filters=filters, kernel_size=4, activation='relu', 
                   kernel_constraint= tf.keras.constraints.MaxNorm(max_value=3, axis=[0,1]))(embedding)
    pool3 = tf.keras.layers.GlobalMaxPool1D()(conv3)
    concat = tf.keras.layers.concatenate([pool1, pool2, pool3], axis=1)
    drop1 = tf.keras.layers.Dropout(0.3)(concat)
    dense1 = tf.keras.layers.Dense(40, activation='relu')(drop1)
    drop2 = tf.keras.layers.Dropout(0.3)(dense1)
    outputs = tf.keras.layers.Dense(5, activation='softmax')(drop2)
    model = tf.keras.Model(inputs=input, outputs=outputs)
    
    # Compile
    model.compile( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
model = glove_cnn(input_dim=vocab_len, output_dim=embed_vector_len, emb_matrix=emb_matrix, max_length=maxLen)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 500)]        0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 500, 100)     16294800    ['input_5[0][0]']                
                                                                                                  
 conv1d_11 (Conv1D)             (None, 499, 30)      6030        ['embedding_4[0][0]']            
                                                                                                  
 conv1d_12 (Conv1D)             (None, 498, 30)      9030        ['embedding_4[0][0]']            
                                                                                            

In [ ]:
model.fit(X_train_indices, y_train, batch_size=256, epochs=100)

Epoch 1/100
166/166 [==============================] - 8s 41ms/step - loss: 1.0898 - accuracy: 0.5963
Epoch 2/100
166/166 [==============================] - 7s 40ms/step - loss: 0.4907 - accuracy: 0.8293
Epoch 3/100
166/166 [==============================] - 7s 40ms/step - loss: 0.3876 - accuracy: 0.8691
Epoch 4/100
166/166 [==============================] - 7s 41ms/step - loss: 0.3314 - accuracy: 0.8914
Epoch 5/100
166/166 [==============================] - 7s 40ms/step - loss: 0.3024 - accuracy: 0.8996
Epoch 6/100
166/166 [==============================] - 7s 40ms/step - loss: 0.2716 - accuracy: 0.9091
Epoch 7/100
166/166 [==============================] - 7s 40ms/step - loss: 0.2540 - accuracy: 0.9151
Epoch 8/100
166/166 [==============================] - 7s 40ms/step - loss: 0.2433 - accuracy: 0.9196
Epoch 9/100
166/166 [==============================] - 7s 40ms/step - loss: 0.2272 - accuracy: 0.9250
Epoch 10/100
166/166 [==============================] - 7s 40ms/step - loss: 0.217

In [ ]:
X_test_indices = tokenizer.texts_to_sequences(X_test)
X_test_indices = tf.keras.preprocessing.sequence.pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [ ]:
y_pred = model.predict(X_test_indices)
y_pred

array([[3.77868625e-09, 1.65965247e-07, 9.99958992e-01, 1.97233696e-08,
        4.08886845e-05],
       [8.37026781e-10, 8.78340511e-09, 1.00000000e+00, 4.47015225e-09,
        5.86022608e-08],
       [1.09656195e-08, 1.96090056e-07, 1.77543704e-08, 9.99999762e-01,
        2.96327087e-08],
       ...,
       [4.88954122e-09, 9.98178363e-01, 1.81123591e-03, 3.37645467e-10,
        1.04404307e-05],
       [2.93833025e-09, 1.10366125e-06, 1.05235749e-05, 9.99988079e-01,
        3.85446185e-07],
       [4.24956514e-09, 4.35618173e-11, 1.86457859e-12, 1.00000000e+00,
        2.43351259e-11]], dtype=float32)

In [ ]:
print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.91      0.90      0.90      2432
           1       0.88      0.81      0.84       489
           2       0.92      0.92      0.92      2504
           3       0.98      0.99      0.99      4171
           4       0.89      0.88      0.89       984

    accuracy                           0.94     10580
   macro avg       0.91      0.90      0.91     10580
weighted avg       0.94      0.94      0.94     10580



In [ ]:
# Analysis on wrong predictions
wrong_indices = np.argmax(y_test, axis=1) != np.argmax(y_pred, axis=1)
wrong_test = y_test[wrong_indices]
wrong_pred = y_pred[wrong_indices]
for i in range(min(wrong_test.shape[0], 100)):
  print('Wrong sample {}'.format(i))
  print(wrong_test[i])
  print(wrong_pred[i])

Wrong sample 0
[0 0 1 0 0]
[9.8210770e-01 2.1860636e-07 1.7891105e-02 4.6731230e-10 9.4680848e-07]
Wrong sample 1
[0 0 1 0 0]
[9.9999976e-01 1.8539986e-13 2.1345197e-07 6.6162543e-16 9.6611946e-12]
Wrong sample 2
[1 0 0 0 0]
[1.8934657e-01 1.5585899e-08 8.0103970e-01 2.7154707e-11 9.6137365e-03]
Wrong sample 3
[1 0 0 0 0]
[4.7547212e-03 1.5318115e-02 9.7958553e-01 1.9848310e-04 1.4315055e-04]
Wrong sample 4
[0 0 0 0 1]
[9.5493615e-01 7.6142926e-05 5.4789896e-05 1.0955580e-08 4.4932965e-02]
Wrong sample 5
[0 0 1 0 0]
[9.5553052e-01 3.3379650e-09 4.4469401e-02 3.7832083e-14 1.2102426e-07]
Wrong sample 6
[0 1 0 0 0]
[0.04184006 0.28882676 0.12673083 0.50715226 0.03545014]
Wrong sample 7
[0 0 1 0 0]
[1.8857248e-04 9.8917633e-01 9.0814782e-03 4.3083855e-04 1.1228339e-03]
Wrong sample 8
[1 0 0 0 0]
[3.4278485e-01 2.8892682e-04 6.5568763e-01 8.9627374e-06 1.2296224e-03]
Wrong sample 9
[1 0 0 0 0]
[2.4096929e-07 1.9284292e-07 9.9999392e-01 5.4639399e-06 1.6169521e-07]
Wrong sample 10
[1 0 0 0 

In [ ]:
model.evaluate(X_test_indices, y_test)

331/331 [==============================] - 3s 7ms/step - loss: 0.2755 - accuracy: 0.9364


[0.27551254630088806, 0.9363893866539001]

In [ ]:
model.save_weights('./cnn_checkpoint')